In [1]:
%load_ext autoreload
%autoreload 2

#### Initialize

In [2]:
from pathlib import Path

import maples_dr
import numpy as np
import pandas as pd
from jppype import Mosaic, vscode_theme

from fundus_vessels_toolkit import AVLabel, FundusData
from fundus_vessels_toolkit.pipelines import FundusAVSegToBiomarkers
from fundus_vessels_toolkit.pipelines.avseg_to_tree import AVSegToTree, NaiveAVSegToTree

vscode_theme()

Thanks for using MAPLES-DR!
  When using this dataset in academic works,
  please cite: ]8;id=764856;https://doi.org/10.1038/s41597-024-03739-6\https://doi.org/10.1038/s41597-024-03739-6]8;;\

HTML(value="<style>\n        .cell-output-ipywidget-background {\n                background: transparent !imp…

In [3]:
maples_dr.configure(messidor_path="/home/gaby/These/Data/Fundus/MESSIDOR/", image_format="rgb")
dataset = maples_dr.load_dataset()

In [4]:
GRAPH_AV_COLORS = {
    AVLabel.BKG: "grey",
    AVLabel.ART: "red",
    AVLabel.VEI: "blue",
    AVLabel.BOTH: "purple",
    AVLabel.UNK: "green",
}

In [5]:
av2tree = AVSegToTree()
naive_av2tree = NaiveAVSegToTree()
av2biomarkers = AVSegToTree()

#### Load image

In [6]:
IMG = Path("20060523_49681_0100_PP.png")  # GOOD
PRED_PATH = Path("/home/gaby/These/src/Fundus/RelatedWorks/AutoMorph/Results/M2/artery_vein/raw")
GT_PATH = Path("/home/gaby/These/Data/Fundus/MAPLES-DR/AV_reviewed/")

AV_PRED = PRED_PATH / IMG.name
sample = dataset[IMG.stem]
fundus = FundusData(
    fundus=sample["fundus"],
    fundus_mask=sample["mask"],
    od=sample["opticDisc"],
    vessels=AV_PRED,
    macula=sample["macula"],
)

av_gt = FundusData.load_vessels(GT_PATH / IMG.name)
v, a = av_gt == AVLabel.ART, av_gt == AVLabel.VEI
av_gt[a] = AVLabel.ART
av_gt[v] = AVLabel.VEI

fundus_gt = FundusData(
    fundus=sample["fundus"],
    fundus_mask=sample["mask"],
    od=sample["opticDisc"],
    vessels=av_gt,
    macula=sample["macula"],
)

## Extract Vascular Graph

In [7]:
av_graph = av2tree.to_vgraph(fundus)

m = Mosaic(2, cols_titles=["Ground Truth", "IA Segmentation"])
fundus_gt.draw(view=m[0], labels_opacity=0.8)
fundus.draw(view=m[1], labels_opacity=0.8)


m[1]["graph"] = av_graph.jppype_layer(bspline=True, node_labels=True)
m[1]["tangents"] = av_graph.geometric_data().jppype_branches_tips_tangents(scaling=10)

m.show()

GridBox(children=(HTML(value='<h3 style="text-align: center;">Ground Truth</h3>'), HTML(value='<h3 style="text…

## Optimize Sub-Trees

In [8]:
lines_digraph_info = av2tree.build_line_digraph(av_graph, fundus)
opti_tree = av2tree.resolve_digraph_to_vtree(*lines_digraph_info)
a_tree, v_tree = av2tree.split_av_tree(opti_tree)

In [10]:
naive_a_tree, naive_v_tree = naive_av2tree(fundus)

In [11]:
def colorize_tree(tree, art, layer):
    if art:
        root_color = "#7a1a1a"
        leaf_color = "#da7676"
        label = AVLabel.ART
    else:
        root_color = "#1a1a7a"
        leaf_color = "#7676da"
        label = AVLabel.VEI
    main_color = GRAPH_AV_COLORS[label]
    nodes_color = pd.Series(main_color, index=tree.nodes_attr.index)
    nodes_color[tree.root_nodes_ids()] = root_color
    nodes_color[tree.leaf_nodes_ids()] = leaf_color
    layer.nodes_cmap = nodes_color.to_dict()
    layer.edges_cmap = main_color


def colorize_subtrees(tree, layer):
    def colormap(x):
        cmap = ["red", "blue", "purple", "green", "orange", "cyan", "pink", "yellow", "teal", "lime"]
        return cmap[x % len(cmap)]

    layer.edges_cmap = pd.Series(tree.subtrees_branch_labels()).map(colormap).to_dict()


m = Mosaic(3, cols_titles=["Naive Tree", "Optimized Tree", "Subtrees"])
fundus_gt.draw(view=m, labels_opacity=0.3)

m[0]["a_tree"] = naive_a_tree.jppype_layer(bspline=True)
m[0]["v_tree"] = naive_v_tree.jppype_layer(bspline=True)
colorize_tree(naive_a_tree, True, m[0]["a_tree"])
colorize_tree(naive_v_tree, False, m[0]["v_tree"])

m[1]["a_tree"] = a_tree.jppype_layer(bspline=True)
m[1]["v_tree"] = v_tree.jppype_layer(bspline=True)
colorize_tree(a_tree, True, m[1]["a_tree"])
colorize_tree(v_tree, False, m[1]["v_tree"])

m[2]["tree"] = opti_tree.jppype_layer(bspline=True)
colorize_subtrees(opti_tree, m[2]["tree"])

m.show()

GridBox(children=(HTML(value='<h3 style="text-align: center;">Naive Tree</h3>'), HTML(value='<h3 style="text-a…

## Generate biomarkers

In [12]:
av2biomarkers = FundusAVSegToBiomarkers(av2tree)

In [13]:
widgets, trees, data = av2biomarkers.inspect_bifurcations(fundus, label_branches=True)
widgets

GridBox(children=(View2D(), View2D()), layout=Layout(grid_template_columns='repeat(2, 1fr)', grid_template_row…

BokehModel(combine_events=True, render_bundle={'docs_json': {'ce3c2d91-05ef-4b28-8f1d-691edec8fbc1': {'version…

In [14]:
widgets, trees, data = av2biomarkers.inspect_branches(fundus)
widgets

GridBox(children=(View2D(), View2D()), layout=Layout(grid_template_columns='repeat(2, 1fr)', grid_template_row…

BokehModel(combine_events=True, render_bundle={'docs_json': {'173649a7-83b3-4520-aa5e-f10bacbb0161': {'version…